In [1]:
%load_ext autoreload
%autoreload 2

from IPython.display import Image
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
import json
import numpy as onp
import jax
import jax.numpy as jnp
import flax
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import timecast as tc

from mpl_toolkits import mplot3d

plt.rcParams['figure.figsize'] = [20, 10]

import tqdm.notebook as tqdm

/home/dsuo/miniconda3/envs/skgaip/lib/python3.7/site-packages/jax/lib/xla_bridge.py:123: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


# Load data

In [68]:
data = onp.array(pd.read_csv("../skgaip/wind/MS_winds.dat", names=list(range(57))))

In [69]:
# Columns = 57 stations
# Rows = wind speed readings (m/s)
data

array([[5.0963, 2.0564, 3.0399, ..., 3.0399, 3.5763, 2.5481],
       [5.0963, 1.5199, 2.5481, ..., 2.5481, 3.5763, 2.5481],
       [5.588 , 1.5199, 2.0564, ..., 2.5481, 3.5763, 1.5199],
       ...,
       [4.6045, 4.0681, 5.0963, ..., 4.6045, 0.    , 3.0399],
       [7.1526, 6.1244, 4.6045, ..., 4.0681, 0.    , 4.0681],
       [7.1526, 3.5763, 3.0399, ..., 4.0681, 0.    , 4.6045]])

In [70]:
# Normalization
# NOTE: This is a bug; they claim this normalizes from 0 to 1, but it doesn't
# NOTE: Their variable also refer to min and max as mean and std, respectively so...
data = (data - data.min()) / data.max()

# Load stored inputs/outputs for verifying

In [189]:
inputs = pickle.load(open("../skgaip/wind/input.pkl", "rb"))
outputs = pickle.load(open("../skgaip/wind/output.pkl", "rb"))
pre = pickle.load(open("../skgaip/wind/pre.pkl", "rb"))
post = pickle.load(open("../skgaip/wind/post.pkl", "rb"))
minmax = pickle.load(open("../skgaip/wind/minmax.pkl", "rb"))
models = pickle.load(open("../skgaip/wind/models.pkl", "rb"))["models"]

# Verify inputs

In [239]:
from timecast.utils import historify

historified = historify(jnp.asarray(data), num_histories=8375, history_len=12)
onp.testing.assert_array_almost_equal(historified, inputs["result"])
onp.testing.assert_array_almost_equal(historified[:6000], inputs["X_train"])
onp.testing.assert_array_almost_equal(data[12:6012,:], inputs["y_train"])
onp.testing.assert_array_almost_equal(historified[6000:6361], inputs["xTest"])
onp.testing.assert_array_almost_equal(data[6012:6373,:], inputs["yTest"])

# Verify predictions

In [233]:
# Copied from https://github.com/amirstar/Deep-Forecast/blob/4dcdf66f8ae3070ab706b30a6e3cf888f36e0536/multiLSTM.py#L210
def predict(X, models):
    results = onp.zeros_like(onp.zeros((X.shape[0], X.shape[2])))
    
    for ind in range(len(X)):
        modelInd = ind % 6
        if modelInd == 0:
            testInputRaw = X[ind]
            testInputShape = testInputRaw.shape
            testInput = onp.reshape(testInputRaw, [1, testInputShape[0], testInputShape[1]])
        else:
            testInputRaw = onp.vstack((testInputRaw, results[ind-1]))
            testInput = onp.delete(testInputRaw, 0, axis=0)
            testInputShape = testInput.shape
            testInput = onp.reshape(testInput, [1, testInputShape[0], testInputShape[1]])
    
        pred = models[modelInd].predict(testInput)
        results[ind] = pred

    return results

In [234]:
pred = predict(inputs["xTest"], models)

In [237]:
onp.testing.assert_array_almost_equal(pred, outputs["pred"])

In [277]:
test_true = data[6012:6373,:] * minmax["max"] + minmax["min"]
test_pred = outputs["pred"] * minmax["max"] + minmax["min"]
jnp.absolute(test_true - test_pred).mean(axis=0).mean()

DeviceArray(1.1508132, dtype=float32)

# Predict all data

In [241]:
historified.shape

(8375, 12, 57)

In [242]:
pred = predict(historified, models)

In [243]:
pred.shape

(8375, 57)

In [293]:
historified.shape

(8375, 12, 57)

In [245]:
true = data[12:]

In [246]:
true.shape

(8375, 57)

In [247]:
jnp.absolute(true - pred).mean(axis=0).mean()

DeviceArray(0.04184432, dtype=float32)